##Exercise #1 ( 6 marks)
Store the file romeo.txt into the databricks DBFS Catalog.

Load the romeo.txt file into a PySpark RDD

Using transformations and actions of RDD, perform a word count. (3 marks)

Output the word which occurs the most amount of times. ( 3 marks)


In [0]:
# load in the Romeo text file
rdd1 = spark.sparkContext.textFile("dbfs:/FileStore/romeo.txt")

# Flatmap to split lines by spaces and return the individual words
rdd2 = rdd1.flatMap(lambda x: x.split(" "))
#print(rdd2.take(5))

# Remove blank characters
rdd3 = rdd2.filter(lambda x: x != "")

# split words into key, value pairs with 1 as the value
rdd4 = rdd3.map(lambda x: (x, 1))

# Reduce to get count of all words and output
rdd5 = rdd4.reduceByKey(lambda a,b: a+b)
print(rdd5.collect())
print()

# Sorting words by number of occurences and outputting the top
rdd6 = rdd5.map(lambda x: (x[1], x[0])).sortByKey(False)
topWord = rdd6.take(1)[0]
print(f"Highest occuring word: '{topWord[1]}' which occured {topWord[0]} times")


[('TRAGEDY', 1), ('OF', 1), ('Shakespeare', 1), ('Dramatis', 1), ('Personae', 1), ('Chorus.', 3), ('Prince', 10), ('of', 369), ('young', 15), ('Count,', 1), ('kinsman', 6), ('Montague,', 14), ('two', 16), ('houses', 2), ('at', 62), ('Capulet', 5), ('Romeo,', 36), ('son', 12), ('nephew', 3), ('Lady', 9), ('Capulet.', 9), ('Friar', 21), ('Laurence,', 1), ('Franciscan.', 2), ('John,', 3), ('Balthasar,', 2), ('Abram,', 1), ('Sampson,', 1), ('Three', 3), ('Musicians.', 2), ('Officer.', 2), ('daughter', 7), ('Nurse', 3), ('Verona;', 2), ('both', 10), ('houses;', 1), ('Torchbearers,', 1), ('Guards,', 1), ('Two', 4), ('alike', 1), ('in', 275), ('dignity,', 1), ('where', 20), ('we', 47), ('scene,', 1), ('ancient', 8), ('grudge', 1), ('new', 13), ('unclean.', 1), ('fatal', 4), ('loins', 1), ('these', 35), ('foes', 1), ("star-cross'd", 1), ('take', 31), ('life;', 2), ('Whose', 4), ('piteous', 4), ('Doth', 12), ("parents'", 2), ('strife.', 1), ('The', 73), ("death-mark'd", 1), ('And', 217), ('cont

## Exercise #2 ( 6 marks)

Choose 3 RDD transformations. 
For each transformation, demonstrate it in a Databricks notebook, leave a comment explaining what the transformation does, and use the take action to ouptut the first 5 entries in the RDD after performing the transformation. (You can use any data you wish to populate the initial RDD before applying each transformation) ( 3 marks)

Choose 3 RDD actions. For each action, demonstrate it in a Databricks notebook, leave a comment explaining what the action does, and if applicable print the results of the action (You can use any data you wish to populate the initial RDD before applying each action) ( 3 marks)



In [0]:
rdd1 = spark.sparkContext.textFile("dbfs:/FileStore/romeo.txt")

### -------- TRANSFORMATIONS -------- ###

# Map. Returns a new dataset formed by passing each element through a given function.
rdd2 = rdd1.map(lambda x: x.split(" "))
print(rdd2.take(5))

# flatMap. Works similar to the map function but each item can be mapped to 0 or more output items. In our example it will take each line and split the words out into one large list, rather than a list of lists.
rdd3 = rdd1.flatMap(lambda x: x.split(" "))
print(rdd3.take(5))

# map again to get into (k, v) pairs for next step
rdd4 = rdd3.map(lambda x: (x, 1))

# reduceByKey. returns a new dataset of (k, v) pairs where the values for each key are aggregated using the given function. In our example it will combine the Words in the keys with the added values of all the 1's in the original (k, v) pairs
rdd5 = rdd4.reduceByKey(lambda a,b: a+b)
print(rdd5.take(5))

### -------- ACTIONS -------- ###

# Count. Very simple, returns the number of elements in the dataset.
print(rdd5.count())

# Collect. Returns all the elements of the dataset as an array.
print(rdd5.collect())

# First. Return the first element of the dataset. Similar to using take(1)
print(rdd5.first())


[['THE', 'TRAGEDY', 'OF', 'ROMEO', 'AND', 'JULIET'], [''], ['by', 'William', 'Shakespeare'], [''], ['Dramatis', 'Personae']]
['THE', 'TRAGEDY', 'OF', 'ROMEO', 'AND']
[('TRAGEDY', 1), ('OF', 1), ('', 16319), ('Shakespeare', 1), ('Dramatis', 1)]
6395
[('TRAGEDY', 1), ('OF', 1), ('', 16319), ('Shakespeare', 1), ('Dramatis', 1), ('Personae', 1), ('Chorus.', 3), ('Prince', 10), ('of', 369), ('young', 15), ('Count,', 1), ('kinsman', 6), ('Montague,', 14), ('two', 16), ('houses', 2), ('at', 62), ('Capulet', 5), ('Romeo,', 36), ('son', 12), ('nephew', 3), ('Lady', 9), ('Capulet.', 9), ('Friar', 21), ('Laurence,', 1), ('Franciscan.', 2), ('John,', 3), ('Balthasar,', 2), ('Abram,', 1), ('Sampson,', 1), ('Three', 3), ('Musicians.', 2), ('Officer.', 2), ('daughter', 7), ('Nurse', 3), ('Verona;', 2), ('both', 10), ('houses;', 1), ('Torchbearers,', 1), ('Guards,', 1), ('Two', 4), ('alike', 1), ('in', 275), ('dignity,', 1), ('where', 20), ('we', 47), ('scene,', 1), ('ancient', 8), ('grudge', 1), ('ne

##Exercise #3 (15 marks)

Eric has started a new business “CNA Wealth Management”. To help them choose stocks to purchase for their customers Eric would like to analyze historical data contained in the NYSE_daily file. The columns in the NYSE_daily file are as follows: (exchange, symbol, date, price open, price high, price low, price close, volume, adjusted price close)

It turns out one stock has an opening value of $0 each day. Please filter out this value so this stock isn’t included. (3 marks)

Using RDD transformations and actions in a PySpark notebok, output each of the following:

1. Which stock had the highest volume of trading on Nov 27th, 2009? (6 marks)

2. Which stock earns the most during Dec 8th, 2009? ( compare the opening and adjusted closing value ) ( 6 marks)

In [0]:
nyse = spark.sparkContext.textFile("dbfs:/FileStore/NYSE_daily")

# split on tabs to get values in lists and remove any rows with an opening value of $0
nyse2 = nyse.map(lambda x: x.split("\t")).filter(lambda x: float(x[3]) != 0)

# filter to get records for Nov 27, 2009
nsye3 = nyse2.filter(lambda x: x[2] == "2009-11-27")

# get new set with stock name and volumes, reduce to get sotck with highest volume
highest_vol = nsye3.map(lambda x: (x[1], int(x[7])))
highest_vol_res = highest_vol.reduce(lambda a, b: a if a[1] > b[1] else b)

print(f"the stock with the highest trade volume on Nov 27th, 2009 was {highest_vol_res[0]} with a volume of {highest_vol_res[1]}")

# filter to get records for Dec 08, 2009
nyse4 = nyse2.filter(lambda x: x[2] == "2009-12-08")

# get new set with stock name and earning, reduce to find stock with highest earning
stock_earnings = nyse4.map(lambda x: (x[1], float(x[8]) - float(x[3])))
highest_stock_earning = stock_earnings.reduce(lambda a, b: a if a[1] > b[1] else b)

print(f"the highest earning stock (comparing the opening and adjusted closing value) for Dec 8th, 2009 was {highest_stock_earning[0]} with a difference of ${round(highest_stock_earning[1], 2)}")


the stock with the highest trade volume on Nov 27th, 2009 was CHK with a volume of 10631100
the highest earning stock (comparing the opening and adjusted closing value) for Dec 8th, 2009 was CLW with a difference of $4.41
